# Snowflake Customers CRUD (Simple)

This notebook connects to Snowflake using environment variables, creates a demo database and a `CUSTOMERS` table, and performs basic CRUD operations.

In [ ]:
# Setup: install connector (if needed)
%pip install snowflake-connector-python snowflake-connector-python[pandas] python-dotenv -q

In [1]:
# 1) Connect to Snowflake using env vars
import os
import snowflake.connector
from dotenv import load_dotenv

load_dotenv()

user = os.getenv("SNOWFLAKE_USER")
password = os.getenv("SNOWFLAKE_PASSWORD")
account = os.getenv("SNOWFLAKE_ACCOUNT")
warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
role = os.getenv("SNOWFLAKE_ROLE")

conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    role=role,
)
cur = conn.cursor()
print("Connected to Snowflake.")

Connected to Snowflake.


In [2]:
# 2) Create DEMO_DB and PUBLIC schema; use them
cur.execute("CREATE DATABASE IF NOT EXISTS DEMO_DB;")
cur.execute("USE DATABASE DEMO_DB;")
cur.execute("CREATE SCHEMA IF NOT EXISTS PUBLIC;")
cur.execute("USE SCHEMA PUBLIC;")
print("Database DEMO_DB and schema PUBLIC are ready.")

Database DEMO_DB and schema PUBLIC are ready.


In [3]:
# 3) Create or replace CUSTOMERS table
cur.execute("DROP TABLE IF EXISTS CUSTOMERS;")
cur.execute(
    """
    CREATE OR REPLACE TABLE CUSTOMERS (
        ID NUMBER(38,0) AUTOINCREMENT,
        NAME VARCHAR(100) NOT NULL,
        EMAIL VARCHAR(200) NOT NULL UNIQUE,
        CREATED_AT TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP
    );
    """
)
print("Table CUSTOMERS created.")

Table CUSTOMERS created.


In [4]:
# 4) CREATE (Insert) sample rows (parameterized)
rows = [
    ("Alice Johnson", "alice@example.com"),
    ("Bob Smith", "bob@example.com"),
    ("Charlie Lee", "charlie@example.com"),
]
cur.executemany(
    "INSERT INTO CUSTOMERS (NAME, EMAIL) VALUES (%(name)s, %(email)s)",
    [{"name": n, "email": e} for n, e in rows]
)
print(f"Inserted {cur.rowcount if cur.rowcount is not None else len(rows)} rows.")

Inserted 3 rows.


In [5]:
# 5) READ (Query) rows - all and filtered
cur.execute("SELECT * FROM CUSTOMERS ORDER BY ID")
all_rows = cur.fetchall()
print("All customers:")
for r in all_rows:
    print(r)

cur.execute("SELECT * FROM CUSTOMERS WHERE ID = %(id)s", {"id": 1})
print("\nFiltered by ID=1:")
print(cur.fetchone())

All customers:
(1, 'Alice Johnson', 'alice@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))
(2, 'Bob Smith', 'bob@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))
(3, 'Charlie Lee', 'charlie@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))

Filtered by ID=1:
(1, 'Alice Johnson', 'alice@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))

Filtered by ID=1:
(1, 'Alice Johnson', 'alice@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))


In [6]:
# 6) UPDATE a row (change email for ID=2)
cur.execute(
    "UPDATE CUSTOMERS SET EMAIL = %(email)s WHERE ID = %(id)s",
    {"email": "bob.new@example.com", "id": 2}
)
print(f"Rows updated: {cur.rowcount}")

cur.execute("SELECT * FROM CUSTOMERS WHERE ID = %(id)s", {"id": 2})
print("Updated row:")
print(cur.fetchone())

Rows updated: 1
Updated row:
(2, 'Bob Smith', 'bob.new@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))
Updated row:
(2, 'Bob Smith', 'bob.new@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))


In [7]:
# 7) DELETE a row (ID=3)
cur.execute("DELETE FROM CUSTOMERS WHERE ID = %(id)s", {"id": 3})
print(f"Rows deleted: {cur.rowcount}")

Rows deleted: 1


In [8]:
# 8) Verify final state and close connection
cur.execute("SELECT COUNT(*) FROM CUSTOMERS")
print("Total remaining:", cur.fetchone()[0])

cur.execute("SELECT * FROM CUSTOMERS ORDER BY ID")
print("Remaining rows:")
for r in cur.fetchall():
    print(r)

# Close connection
cur.close()
conn.close()
print("Snowflake connection closed.")

# Optional cleanup (uncomment to drop the table)
# cur = conn.cursor()
# cur.execute("DROP TABLE IF EXISTS CUSTOMERS")
# cur.close()

Total remaining: 2
Remaining rows:
(1, 'Alice Johnson', 'alice@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))
(2, 'Bob Smith', 'bob.new@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))
Snowflake connection closed.
Remaining rows:
(1, 'Alice Johnson', 'alice@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))
(2, 'Bob Smith', 'bob.new@example.com', datetime.datetime(2025, 10, 13, 16, 35, 32, 847000))
Snowflake connection closed.
